# From a MNIST handwritten digit (SVM) classifier to Apple's Core ML format

This tutorial will not go in depth of machine learning topics, it is a short tutorial on how to convert scikit-learn SVM model Apple's coreml format. 

In [3]:
import mnist_loader
from sklearn import svm

# 1.  Loading the MNIST image data

First we will load the MNIST image data. Here, we do this with the help of an external library (mnist_loader).

You can find the original python file in the following github repo: https://github.com/mnielsen/neural-networks-and-deep-learning/tree/master/src

In [17]:
training_data, validation_data, test_data = mnist_loader.load_data()

We will not be using the validation data, however, we kept the standard way of data partitining.

Training data is returned as a tuple of two entires. The first entry contains the actual training images in a vectorized form (two dimensional 28 pixels by 28 pixels image is flattend to 1 x 784 sized vector). In total, we have 50K traning images, hence, 50K x 784 sized training data. The second entry contains the true label (in this case a digid between 0 and 9) of each training image. 

In [16]:
print (type(training_data))
print ("First entry: ", training_data[0].shape)
print ("Second entry: ", training_data[1].shape)
print ("Indeed, each entry of training data is numpy ndarray:")
print (type(training_data[0]))
print (type(training_data[1]))

<type 'tuple'>
('First entry: ', (50000, 784))
('Second entry: ', (50000,))
Indeed, each entry of training data is numpy ndarray:
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [14]:
print ("true label of the first image is : ", training_data[1][0])

('label of the first image is : ', 5)


Test data has the same structure, however, only with 10K images.

In [19]:
print (type(test_data))
print ("First entry: ", test_data[0].shape)
print ("Second entry: ", test_data[1].shape)
print (type(test_data[0]))
print (type(test_data[1]))

<type 'tuple'>
('First entry: ', (10000, 784))
('Second entry: ', (10000,))
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


# 2. Train the model

Before going to model training, I will rename the trainining and test data components.

In [27]:
X_train = training_data[0]
y_train = training_data[1]
X_test = test_data[0]
y_test = test_data[1]

We use the Support Vector Machines (SVM) from scikit-learn library to build our ML model. I will use only first 1000 data images for fitting in order to quickly check if the model is built successfully.

In [28]:
X_train_sub = X_train[:1000]
y_train_sub = y_train[:1000]
svm_model = svm.SVC()
svm_model.fit(X_train_sub, y_train_sub)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Now, we have SVM model is trained. We can immediately check what will be the prediction of the model for the first digit in the test data.

In [29]:
predicted_test_0 = svm_model.predict(X_test[0])
print("predicted: ", int(predicted_test_0), "true value: ", y_test[0])

('predicted: ', 7, 'true value: ', 7)


//anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


You should see 7 for the predicted and true label of the digid. Even if we used 1K data out of 50K data, our model was good enough to predict the first digid.

We can check the performance of the model on whole test data.

In [30]:
# testing
predictions = [int(a) for a in svm_model.predict(X_test)]
num_correct = sum(int(a == y) for a, y in zip(predictions, y_test))
print("SVM classifier (trained with subset of training data): ")
print("%s of %s values correct." % (num_correct, len(y_test)))

SVM classifier (trained with subset of training data): 
8267 of 10000 values correct.


You should see the performance around 82 %. That is not bad, however, we can do better if we use all training data for building the model. This can take a while to finish.

In [31]:
svm_model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Performance on the test data should be increased to around 94 %. 

In [32]:
# testing
predictions = [int(a) for a in svm_model.predict(X_test)]
num_correct = sum(int(a == y) for a, y in zip(predictions, y_test))
print("SVM classifier (trained with subset of training data): ")
print("%s of %s values correct." % (num_correct, len(y_test)))

SVM classifier (trained with subset of training data): 
9435 of 10000 values correct.


# 3. Converting SVM model to Core ML format

In [44]:
import coremltools

In [34]:
# we will use coremltools's converter, first we can check signature of the convert function: 
coremltools.converters.sklearn.convert?

In [36]:
coreml_svm = coremltools.converters.sklearn.convert(svm_model, 'hand_digit', 'predicted_digit')
# save the model
coreml_svm.save('digit_recognizer.mlmodel')

The coreml model should be visible in the directory where you have this notebook. Optionally, you can also add more information/metadata to your model.

In [37]:
# set model metadata
coreml_svm.author = 'Erkan Diken'
coreml_svm.license = 'BSD'
coreml_svm.short_description = 'Scikit-learn SVM classifier based handwritten digit recognizer '

# set feature descriptions manually
coreml_svm.input_description['hand_digit'] = 'handwritten digit'

# set the output descriptions
coreml_svm.output_description['predicted_digit'] = 'predicted digit'

# save the model
coreml_svm.save('digit_recognizer.mlmodel')

# 4. Load the Core ML model to make predictions 
Note: If you do not have MacOS version of 10.13. The predict method on model will throw the following exception: Model prediction is only supported on macOS version 10.13.

In [39]:
# load the model and predict the first digid in test data set
model = coremltools.models.MLModel('digit_recognizer.mlmodel')
#print (int(model.predict(X_test[0]))) 

However, it is still possible to see the specs of your model.

In [40]:
model.get_spec

<bound method MLModel.get_spec of input {
  name: "hand_digit"
  shortDescription: "handwritten digit"
  type {
    multiArrayType {
      shape: 784
    }
  }
}
output {
  name: "predicted_digit"
  shortDescription: "predicted digit"
  type {
    int64Type {
    }
  }
}
predictedFeatureName: "predicted_digit"
metadata {
  shortDescription: "Scikit-learn SVM classifier based handwritten digit recognizer "
  author: "Erkan Diken"
  license: "BSD"
}
>